In [2]:
import sqlite3
import pandas as pd
import numpy as np
import plotly.express as px

In [98]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect("../db/hr.db")
df = pd.read_sql_query("SELECT * from base_info", con)

In [99]:
columns = ['기준일자','회사','사번','근무지']

In [100]:
df1 = df[columns]
df1.head(2)

,기준일자,회사,사번,근무지
0,t20210801,HCE,A421197,분당사무소
1,t20210801,HCE,A510951,분당사무소


In [101]:
loca = pd.read_excel("location.xlsx")
loca

,근무지,위도,경도
0,GRC,37.376923,127.106108
1,경주,35.946756,129.267161
2,군산,35.969777,126.596542
3,군산공장,35.967714,126.250069
4,보령Proving Ground,36.285188,126.637155
5,분당두산타워,37.364096,127.000353
6,분당사무소,37.378442,127.214230
7,안산부품센터,37.285315,126.888282
8,용인연구소,37.306574,127.137412
9,울산본사,35.505778,129.441743


In [102]:
target_df = df1[df1["회사"]!="HCM"]
# target_df = df1
target_df

,기준일자,회사,사번,근무지
0,t20210801,HCE,A421197,분당사무소
1,t20210801,HCE,A510951,분당사무소
2,t20210801,HCE,A510197,분당사무소
3,t20210801,HCE,A460222,분당사무소
4,t20210801,HCE,A321109,분당사무소
...,...,...,...,...
38518,t20230401,HDI,1926449,인천공장
38519,t20230401,HDI,1925478,인천공장
38520,t20230401,HDI,1925657,인천공장
38521,t20230401,HDI,1925909,인천공장


In [103]:
target_df = pd.merge(target_df,loca, on=['근무지'], how='left')
target_df

,기준일자,회사,사번,근무지,위도,경도
0,t20210801,HCE,A421197,분당사무소,37.378442,127.214230
1,t20210801,HCE,A510951,분당사무소,37.378442,127.214230
2,t20210801,HCE,A510197,분당사무소,37.378442,127.214230
3,t20210801,HCE,A460222,분당사무소,37.378442,127.214230
4,t20210801,HCE,A321109,분당사무소,37.378442,127.214230
...,...,...,...,...,...,...
38457,t20230401,HDI,1926449,인천공장,37.486461,126.629820
38458,t20230401,HDI,1925478,인천공장,37.486461,126.629820
38459,t20230401,HDI,1925657,인천공장,37.486461,126.629820
38460,t20230401,HDI,1925909,인천공장,37.486461,126.629820


In [104]:
target_df1 = target_df.groupby(["근무지","기준일자","회사", "위도", "경도"])[["사번"]].count().reset_index()
target_df1 = target_df1.rename(columns = {"사번": "인원"})
# target_df1["num"] = target_df1["사번"]*2
target_df1

,근무지,기준일자,회사,위도,경도,인원
0,GRC,t20220101,HDI,37.376923,127.106108,1
1,GRC,t20220401,HDI,37.376923,127.106108,1
2,GRC,t20220701,HDI,37.376923,127.106108,1
3,GRC,t20221001,HCE,37.376923,127.106108,1
4,GRC,t20221001,HDI,37.376923,127.106108,1
...,...,...,...,...,...,...
138,인천공장,t20220401,HDI,37.486461,126.629820,2067
139,인천공장,t20220701,HDI,37.486461,126.629820,2037
140,인천공장,t20221001,HDI,37.486461,126.629820,2030
141,인천공장,t20230101,HDI,37.486461,126.629820,1663


In [105]:
def log_trans(x):
    return np.log10(x)

In [106]:
target_df1["로그"] = target_df1["인원"].apply(log_trans)
target_df1

,근무지,기준일자,회사,위도,경도,인원,로그
0,GRC,t20220101,HDI,37.376923,127.106108,1,0.000000
1,GRC,t20220401,HDI,37.376923,127.106108,1,0.000000
2,GRC,t20220701,HDI,37.376923,127.106108,1,0.000000
3,GRC,t20221001,HCE,37.376923,127.106108,1,0.000000
4,GRC,t20221001,HDI,37.376923,127.106108,1,0.000000
...,...,...,...,...,...,...,...
138,인천공장,t20220401,HDI,37.486461,126.629820,2067,3.315340
139,인천공장,t20220701,HDI,37.486461,126.629820,2037,3.308991
140,인천공장,t20221001,HDI,37.486461,126.629820,2030,3.307496
141,인천공장,t20230101,HDI,37.486461,126.629820,1663,3.220892


In [107]:
order1= {'기준일자':['t20210801', 't20211001', 't20220101', 't20220401', 't20220701', 't20221001', 't20230101', 't20230401']}

In [117]:
px.set_mapbox_access_token(open("mapbox_token.py").read())
df = target_df1
fig = px.scatter_mapbox(df, lat="위도", lon="경도",  color="회사", size="로그",
                        color_continuous_scale=px.colors.sequential, size_max=60, zoom=7,
                        center = {'lat':36.40, 'lon':128.10}, height=800, mapbox_style='light',
                        labels = {'회사': 'company'}, animation_frame="기준일자",
                        animation_group="근무지", category_orders=order1,
                        hover_name="근무지", opacity=0.5, hover_data=["기준일자", "인원"], text="회사")
fig.show()

2023-05-03 07:53:45.442 
  command:

    streamlit run C:\hr\hrenv\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [8]:
a = np.random.random(1)/100
a[0]

0.009980002836602657